In [1]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from scipy.stats import ks_2samp, chi2_contingency
import concurrent.futures


In [2]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [12]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [18]:
X_train[0][6]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.11764706, 0.14117647,
       0.36862745, 0.60392157, 0.66666667, 0.99215686, 0.99215686,
       0.99215686, 0.99215686, 0.99215686, 0.88235294, 0.6745098 ,
       0.99215686, 0.94901961, 0.76470588, 0.25098039, 0.        ,
       0.        , 0.        , 0.        ])

In [19]:
# Normalize the data
X_train, X_test = X_train / 255.0, X_test / 255.0

In [20]:
X_train[0][6]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00046136, 0.00055363,
       0.0014456 , 0.00236832, 0.00261438, 0.00389081, 0.00389081,
       0.00389081, 0.00389081, 0.00389081, 0.00346021, 0.00264514,
       0.00389081, 0.00372165, 0.00299885, 0.00098424, 0.        ,
       0.        , 0.        , 0.        ])

In [4]:
# Define a model creation function
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

####  We have to convert our keras model to tff model
  This is necessary because TFF operates on its own set of model interfaces that are designed to be compatible with federated computations

In a federated learning setting, the model needs to be serialized and sent to multiple clients, where it will be used to compute updates locally.
These updates are then sent back to the server where they are aggregated. 
This process requires the model to have certain properties and methods that are not present in a standard Keras model.

In [23]:
# Convert Keras model to TFF model
def model_fn():
    keras_model = create_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        
        #means that the input specification is based on the first client dataset. This assumes that all client datasets have the same format
        #used to specify the type and shape of the data that the model expects for its inputs
        #The element_spec property of a tf.data.Dataset object gives the shape and type information of the dataset elements
        
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )
    
# The tff.learning.from_keras_model function takes a Keras model and wraps it in a TFF model interface,
# which includes methods for sending the model to clients,
# computing updates, and aggregating updates on the server.
# It also includes methods for training, evaluation, and prediction that can be used in a federated setting.    

In [24]:
# Set the number of clients in the federated learning system
num_clients = 10

# Initialize an empty list to hold the datasets for each client
client_datasets = []

# Create a dataset for each client
for _ in range(num_clients):
    # Generate 50 random 28x28 images (values between 0 and 1)
    images = np.random.rand(50, 28, 28)
    
    # Generate 50 random labels (integers between 0 and 9)
    labels = np.random.randint(0, 10, 50)
    
    # Create a dataset from the images and labels
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    # Batch the dataset into groups of 10
    batched_dataset = dataset.batch(10)
    #.batch(10): This batches the dataset into batches of 10 elements each.
    # This means that during training, the model will be updated based on 10 examples at a time.
    
    # Add the batched dataset to the list of client datasets
    client_datasets.append(batched_dataset)


In [6]:
# Create dummy client datasets (reduced size)
# num_clients = 10
# client_datasets = [tf.data.Dataset.from_tensor_slices((np.random.rand(50, 28, 28), np.random.randint(0, 10, 50))).batch(10) for _ in range(num_clients)]

this part of the code is creating a list of federated learning algorithms, one for each client in the system

In [27]:
# Create federated learning algorithms for each client
federated_algorithms = [tff.learning.build_federated_averaging_process(
    # This function is used to build a Federated Averaging process in TensorFlow Federated
    
    model_fn,
    # means that this model will be used for the federated learning process
    
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    # This is a function that returns an optimizer for updating the model parameters on the clients
    
    
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
    # This is a function that returns an optimizer for updating the global model parameters on the server
    
) for _ in range(num_clients)]


len(federated_algorithms)

10

In [29]:

def initialize_algorithm(algorithm):
    tff.backends.native.set_local_execution_context()
    return algorithm.initialize()


with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(initialize_algorithm, federated_algorithms[0])
    state = future.result()

print('Initial state:', state)


Initial state: ServerState(model=ModelWeights(trainable=[array([[ 0.02600021, -0.04608943,  0.0248811 , ...,  0.04855453,
         0.04891501, -0.0463436 ],
       [ 0.00033134, -0.05374972, -0.03009736, ...,  0.03900046,
         0.04702925, -0.0004184 ],
       [ 0.00073104,  0.01230446, -0.07704816, ..., -0.02358056,
        -0.00164101, -0.00112464],
       ...,
       [-0.02896514, -0.0526791 ,  0.05669478, ...,  0.07898881,
        -0.05068234, -0.08029824],
       [-0.0772846 , -0.03591786,  0.06588206, ..., -0.03342385,
        -0.06693491, -0.03967508],
       [ 0.06169649, -0.03959437, -0.05611445, ...,  0.03519689,
        -0.05067812,  0.04298324]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.,

model: This is the initial model weights. It’s divided into trainable and non_trainable weights. The trainable weights are the ones that the algorithm will update during training. In this case, they are initialized with random values.

optimizer_state: This is the state of the optimizer. It can include things like the current step number, momentum variables for optimizers like Adam or RMSProp, etc. In this case, it’s just the current step number, initialized to 0.

delta_aggregate_state and model_broadcast_state: These are used by the server to aggregate the model updates from the clients (delta_aggregate_state) and broadcast the updated model back to the clients (model_broadcast_state). In this case, they are both empty because no updates have been performed yet.

In [8]:
# Function to run the training in a separate event loop
def run_training():
    
    tff.backends.native.set_local_execution_context()
    # This sets up the execution context for TensorFlow Federated (TFF) in the local machine

    states = [algorithm.initialize() for algorithm in federated_algorithms]
    # This initializes the state of each federated learning algorithm
    # The state includes the model parameters and any other variables that the algorithm needs to keep track of

    # Train the models on the clients' data (reduced to 2 rounds)
    for round_num in range(1, 3):
        print(f"Round {round_num}")
        for client_id in range(num_clients):
            states[client_id], _ = federated_algorithms[client_id].next(states[client_id], [client_datasets[client_id]])

        # Perform differential testing for each pair of clients
        for i in range(num_clients):
            for j in range(i+1, num_clients):
                # Get the model weights after training
                weights_i = states[i].model.trainable
                weights_j = states[j].model.trainable

                # Evaluate both models on train and test data
                train_predictions_i, train_labels = evaluate_model_on_data(weights_i, create_model, X_train[:1000], Y_train[:1000])
                test_predictions_i, test_labels = evaluate_model_on_data(weights_i, create_model, X_test[:1000], Y_test[:1000])
                train_predictions_j, _ = evaluate_model_on_data(weights_j, create_model, X_train[:1000], Y_train[:1000])
                test_predictions_j, _ = evaluate_model_on_data(weights_j, create_model, X_test[:1000], Y_test[:1000])

                # Perform differential testing
                print(f"Comparison between Client {i} and Client {j}:")
                perform_differential_testing(train_predictions_i, train_predictions_j, train_labels, "Train")
                perform_differential_testing(test_predictions_i, test_predictions_j, test_labels, "Test")
                print()

        # Average the predictions after differential testing
        average_predictions_train = np.mean([evaluate_model_on_data(state.model.trainable, create_model, X_train[:1000], Y_train[:1000])[0] for state in states], axis=0)
        average_predictions_test = np.mean([evaluate_model_on_data(state.model.trainable, create_model, X_test[:1000], Y_test[:1000])[0] for state in states], axis=0)

        print("Average predictions on Train Data:")
        print(average_predictions_train)
        print("Average predictions on Test Data:")
        print(average_predictions_test)

In [9]:
# Function to evaluate the model on data
def evaluate_model_on_data(weights, create_model_fn, X, Y):
    model = create_model_fn()
    model.set_weights(weights)
    predictions = model.predict(X)
    return predictions, Y

In [10]:
# Function to perform differential testing
def perform_differential_testing(predictions_i, predictions_j, labels, data_type):
    # Criterion 1: Absolute differences between classes
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    Δ_class = np.sum(pred_class_i != pred_class_j)

    # Criterion 2: Absolute differences between scores
    Δ_score = np.sum(predictions_i != predictions_j)

    # Criterion 3: Significance of difference between scores
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue

    # Criterion 4: Significance of difference between classifications
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    print(f"{data_type} Data:")
    print(f"Δ_class: {Δ_class}")
    print(f"Δ_score: {Δ_score:.2f}")
    print(f"P_KS: {P_KS:.4f}")
    print(f"P_X2: {P_X2:.4f}")
    
    if P_KS < 0.05 or P_X2 < 0.05:
        print("Warning: Significant difference detected (p-value < 0.05)")

In [11]:
# Run the training
with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(run_training)
    future.result()


Round 1
Comparison between Client 0 and Client 1:
Train Data:
Δ_class: 803
Δ_score: 10000.00
P_KS: 0.0076
P_X2: 0.0000
Test Data:
Δ_class: 858
Δ_score: 10000.00
P_KS: 0.1230
P_X2: 0.0000

Comparison between Client 0 and Client 2:
Train Data:
Δ_class: 884
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000
Test Data:
Δ_class: 861
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000

Comparison between Client 0 and Client 3:
Train Data:
Δ_class: 883
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000
Test Data:
Δ_class: 897
Δ_score: 10000.00
P_KS: 0.0116
P_X2: 0.0000

Comparison between Client 0 and Client 4:
Train Data:
Δ_class: 855
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000
Test Data:
Δ_class: 870
Δ_score: 10000.00
P_KS: 0.1189
P_X2: 0.0000

Comparison between Client 0 and Client 5:
Train Data:
Δ_class: 980
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000
Test Data:
Δ_class: 976
Δ_score: 10000.00
P_KS: 0.0000
P_X2: 0.0000

Comparison between Client 0 and Client 6:
Train Data:
Δ_class: 936
Δ_score: 10000.00
P_KS: 0